In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from collections import defaultdict
from scipy.stats import linregress
from datetime import timedelta

In [2]:
projects = [
    #'rdatatable_datatable',
    #'facebook_react',
    'freecad_freecad'
]

In [3]:
issues = list()
issues_comments = list()
pull_requests = list()
pull_request_comments = list()

for file in projects:
    temp_df = pd.read_excel(f'Files/{file}_issues.xlsx')
    temp_df['project'] = file
    issues.append(temp_df)

    temp_df = pd.read_excel(f'Files/{file}_issues_comments.xlsx')
    temp_df['project'] = file
    issues_comments.append(temp_df)

    temp_df = pd.read_excel(f'Files/{file}_pull_requests.xlsx')
    temp_df['project'] = file
    pull_requests.append(temp_df)

    temp_df = pd.read_excel(f'Files/{file}_pull_request_comments.xlsx')
    temp_df['project'] = file
    pull_request_comments.append(temp_df)

In [4]:
df_issues = pd.concat(issues)
df_issues_comments = pd.concat(issues_comments)
df_pull_requests = pd.concat(pull_requests)
df_pull_requests_comments = pd.concat(pull_request_comments)

In [5]:
df_issues['created_by'] = df_issues['created_by']\
    .str.replace('https://api.github.com/users/', '', regex = False)

df_pull_requests['created_by'] = df_pull_requests['created_by']\
    .str.replace('https://api.github.com/users/', '', regex = False)

df_issues_comments['created_by'] = df_issues_comments['created_by']\
    .str.extract(r'login="([^"]+)"')

df_pull_requests_comments['created_by'] = df_pull_requests_comments['created_by']\
    .str.extract(r'login="([^"]+)"')

In [6]:
dim_users = pd.concat([
        df_issues[['created_by', 'project']],
        df_pull_requests[['created_by', 'project']],
        df_issues_comments[['created_by', 'project']],
        df_pull_requests_comments[['created_by', 'project']]
    ])\
    .drop_duplicates()\
    .reset_index()\
    [['created_by', 'project']]

In [7]:
pr_reviewers_by_month = df_pull_requests_comments[['created_by', 'created_at', 'pull_request_id', 'project']]
pr_reviewers_by_month['created_at'] = pd.to_datetime(pr_reviewers_by_month['created_at'], errors='coerce').dt.strftime('%Y-%m')
pr_reviewers_by_month = pr_reviewers_by_month.drop_duplicates()
pr_reviewers_by_month = pr_reviewers_by_month.groupby(['created_at', 'created_by', 'project']).count().reset_index()
pr_reviewers_by_month = pr_reviewers_by_month.rename(columns={'pull_request_id': 'number_of_revisions'})

/var/folders/c1/3ypglz8x6bx6_n9y6760l6n9h4gtsy/T/ipykernel_40368/2069917712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_reviewers_by_month['created_at'] = pd.to_datetime(pr_reviewers_by_month['created_at'], errors='coerce').dt.strftime('%Y-%m')


In [8]:
issues_commented = df_issues_comments[['created_by', 'created_at', 'issue_id', 'project']]
issues_commented['created_at'] = pd.to_datetime(issues_commented['created_at'], errors='coerce').dt.strftime('%Y-%m')
issues_commented = issues_commented.drop_duplicates()
issues_commented = issues_commented.groupby(['created_at', 'created_by', 'project']).count().reset_index()
issues_commented = issues_commented.rename(columns={'issue_id': 'number_of_comments_issues'})

/var/folders/c1/3ypglz8x6bx6_n9y6760l6n9h4gtsy/T/ipykernel_40368/1794930000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issues_commented['created_at'] = pd.to_datetime(issues_commented['created_at'], errors='coerce').dt.strftime('%Y-%m')


In [9]:
issues_created = df_issues[['created_by', 'created_at', 'id', 'project']]
issues_created['created_at'] = pd.to_datetime(issues_created['created_at'], errors='coerce').dt.strftime('%Y-%m')
issues_created = issues_created.drop_duplicates()
issues_created = issues_created.groupby(['created_at', 'created_by', 'project']).count().reset_index()
issues_created = issues_created.rename(columns={'id': 'number_of_issues'})

/var/folders/c1/3ypglz8x6bx6_n9y6760l6n9h4gtsy/T/ipykernel_40368/2771644957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issues_created['created_at'] = pd.to_datetime(issues_created['created_at'], errors='coerce').dt.strftime('%Y-%m')


In [10]:
pull_request_created = df_pull_requests[['created_by', 'created_at', 'id', 'project']]
pull_request_created['created_at'] = pd.to_datetime(pull_request_created['created_at'], errors='coerce').dt.strftime('%Y-%m')
pull_request_created = pull_request_created.drop_duplicates()
pull_request_created = pull_request_created.groupby(['created_at', 'created_by', 'project']).count().reset_index()
pull_request_created = pull_request_created.rename(columns={'id': 'number_of_pr'})

/var/folders/c1/3ypglz8x6bx6_n9y6760l6n9h4gtsy/T/ipykernel_40368/605893098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pull_request_created['created_at'] = pd.to_datetime(pull_request_created['created_at'], errors='coerce').dt.strftime('%Y-%m')


In [11]:
df_issues_interaction = pd.concat([
        df_issues_comments[['issue_id', 'created_by', 'created_at', 'project']],
        df_issues[['id', 'created_by', 'created_at', 'project']].rename(columns = {'id': 'issue_id'})
    ], ignore_index=True)

df_pr_interaction = pd.concat(
    [
        df_pull_requests_comments[['pull_request_id', 'created_by', 'created_at', 'project']],
        df_pull_requests[['id', 'created_by', 'created_at', 'project']].rename(columns = {'id': 'pull_request_id'})
    ],
    ignore_index=True
)

df_issues_interaction['object'] = 'Issue'
df_pr_interaction['object'] = 'PullRequest'

df_interaction = pd.concat([
        df_pr_interaction.rename(columns = {'pull_request_id': 'id'}),
        df_issues_interaction.rename(columns = {'issue_id': 'id'})
    ],
    ignore_index=True
)

df_interaction['created_at'] = pd.to_datetime(df_interaction['created_at']).dt.strftime('%Y-%m')

In [12]:
df_interaction = df_interaction\
    .sort_values(by = ['object', 'created_at'])\
    .reset_index(drop=True)

interactions = []
interaction_points = defaultdict(int)

for project in df_interaction['project'].unique():
    temp_df = df_interaction[df_interaction['project'] == project]
    
    for obj in temp_df['object'].unique():
        obj_df = temp_df[temp_df['object'] == obj]

        for page in obj_df['id'].unique():
            page_df = obj_df[obj_df['id'] == page]
            
            previous_users = set()

            for _, row in page_df.iterrows():
                current_user = row['created_by']
                created_at = row['created_at']
                
                for user in previous_users:
                    interaction_points[(current_user, user, created_at, project)] += 1
                
                previous_users.add(current_user)

In [13]:
interactions_df = pd.DataFrame(
    [(dev_a, dev_b, created_at, project, points) for (dev_a, dev_b, created_at, project), points in interaction_points.items()],
    columns=['Developer_A', 'Interacted_With', 'Created_At', 'Project', 'Points']
)

In [14]:
aggregated = interactions_df\
    .groupby(['Developer_A', 'Interacted_With', 'Created_At', 'Project'])\
    .agg(Total_Interactions_A_to_B=('Points', 'sum'))\
    .reset_index()

reverse_interactions = interactions_df\
    .rename(columns={'Developer_A': 'Interacted_With', 'Interacted_With': 'Developer_A'})\
    .groupby(['Developer_A', 'Interacted_With', 'Created_At', 'Project'])\
    .agg(Total_Interactions_B_to_A = ('Points', 'sum'))\
    .reset_index()

df = pd.merge(
        aggregated, 
        reverse_interactions, 
        on = ['Developer_A', 'Interacted_With', 'Created_At', 'Project'], 
        how = 'outer'
    ).fillna(0)

df['Relationship_Strength'] = df[['Total_Interactions_A_to_B', 'Total_Interactions_B_to_A']].min(axis=1)
df = df[df['Developer_A'] != df['Interacted_With']]
df = df.rename(columns={'Interacted_With': 'Developer_B', 'Created_At': 'Date'})

In [15]:
monthly_metrics = []

for project in df['Project'].drop_duplicates():
    
    temp_df = df[df['Project'] == project]
    
    for month, month_df in temp_df.groupby(temp_df['Date']):
        
        G = nx.Graph()
        
        for _, row in month_df.iterrows():
            G.add_edge(
                row['Developer_A'], 
                row['Developer_B'], 
                weight=row['Relationship_Strength']
            )
        
        degree_centrality = nx.degree_centrality(G)
        betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
        closeness_centrality = nx.closeness_centrality(G)

        for user in G.nodes():
            user_edges = list(G.edges(user, data=True))
            num_relationships = len(user_edges)
            
            avg_strength = (
                sum(edge_data['weight'] for _, _, edge_data in user_edges) / num_relationships
                if num_relationships > 0 else 0
            )

            monthly_metrics.append({
                'user': user,
                'month': month,
                'degree_centrality': degree_centrality.get(user, 0),
                'betweenness_centrality': betweenness_centrality.get(user, 0),
                'closeness_centrality': closeness_centrality.get(user, 0),
                'num_relationships': num_relationships,
                'avg_strength': avg_strength,
                'project': project,
            })


    network_df = pd.DataFrame(monthly_metrics)

In [16]:
df = dim_users\
    .merge(
        pr_reviewers_by_month,
        on=['created_by', 'project'], 
        how='outer'
    )\
    .merge(
        issues_created,
        on=['created_by', 'project', 'created_at'], 
        how='outer'
    )\
    .merge(
        issues_commented,
        on=['created_by', 'project', 'created_at'], 
        how='outer'
    )\
    .merge(
        pull_request_created,
        on=['created_by', 'project', 'created_at'], 
        how='outer'
    )\
    .rename(
        columns = {
            'created_by': 'user',
            'created_at': 'month'
        }
    )\
    .merge(
        network_df,
        on=['user', 'month', 'project']
    )

In [17]:
columns_to_fill = [
    'number_of_revisions',
    'number_of_issues',
    'number_of_comments_issues',
    'number_of_pr',
    'degree_centrality',
    'betweenness_centrality',
    'closeness_centrality',
    'num_relationships',
    'avg_strength',
]

df[columns_to_fill] = df[columns_to_fill].fillna(0)

In [18]:
all_rows = []

for (developer, project), group in df.groupby(['user', 'project']):
    group = group.sort_values(by='month', ascending=True) 
    
    for idx, current_month in enumerate(group['month']):
    
        full_months = pd.date_range(start=current_month, periods=13, freq='MS')
        full_months_period = full_months.to_period('M')
        
        existing_months_period = group['month']
        missing_months = full_months_period.difference(existing_months_period)
    
        if not missing_months.empty:
            missing_data = pd.DataFrame({
                'user': developer,
                'project': project,
                'month':  pd.to_datetime(missing_months.start_time).strftime('%Y-%m'),
                'number_of_revisions': 0,
                'number_of_issues': 0,
                'number_of_comments_issues': 0,
                'number_of_pr': 0,
                'degree_centrality': 0,
                'betweenness_centrality': 0,
                'closeness_centrality': 0,
                'num_relationships': 0,
                'avg_strength': 0
            })
            all_rows.append(missing_data)

if all_rows:
    missing_months_df = pd.concat(all_rows, ignore_index=True)
    df = pd\
        .concat([df, missing_months_df], ignore_index=True)\
        .drop_duplicates(subset=['user', 'project', 'month'])

df = df.sort_values(by=['user', 'project', 'month']).reset_index(drop=True)

In [19]:
regression_results = []

for (developer, project), group in df.groupby(['user', 'project']):
    group = group.sort_values(by='month', ascending=False)
    group['month'] = pd.to_datetime(group['month'])
    group['month_number'] = np.arange(len(group))

    for idx, current_month in enumerate(group['month']):
        current_month = pd.to_datetime(current_month)
        
        full_months = pd.date_range(end=current_month, periods=13, freq='MS')
        group['month_year'] = group['month'].dt.to_period('M')
        full_months_period = full_months.to_period('M')
        missing_months = full_months_period.difference(group['month_year'])
        
        if not missing_months.empty:
            missing_data = pd.DataFrame({
                'month': missing_months.start_time,
                'user': developer,
                'project': project,
                'month_number': np.arange(len(group), len(group) + len(missing_months)),
                'number_of_revisions': 0,
                'number_of_issues': 0,
                'number_of_comments_issues': 0,
                'number_of_pr': 0,
                'degree_centrality': 0,
                'betweenness_centrality': 0,
                'closeness_centrality': 0,
                'num_relationships': 0,
                'avg_strength': 0
            })
            group = pd.concat([group, missing_data], ignore_index=True)

        for window in [3, 6, 9, 12]:
            if len(group) < window:
                continue
            
            window_data = group.tail(window)
            x = window_data['month_number']
            
            for metric in [
                'number_of_revisions', 
                'number_of_issues', 
                'number_of_comments_issues', 
                'number_of_pr', 
                'degree_centrality', 
                'betweenness_centrality', 
                'closeness_centrality',
                'num_relationships', 
                'avg_strength'
            ]:
                y = window_data[metric]
                slope, intercept, r_value, p_value, std_err = linregress(x, y)
                
                predicted_y = slope * x + intercept
                residuals = y - predicted_y
                std_dev = np.std(residuals)
                
                result = {
                    'user': developer,
                    'project': project,
                    'current_month': current_month,
                    f'{metric}_{window}_slope': slope,
                    f'{metric}_{window}_intercept': intercept,
                    f'{metric}_{window}_std_dev': std_dev
                }
                
                regression_results.append(result)

regression_df = pd.DataFrame(regression_results)

regression_df_pivot = regression_df.pivot_table(
    index=['user', 'project', 'current_month'],
    aggfunc='first'
)

regression_df_pivot.columns = [f'{col}' for col in regression_df_pivot.columns]
regression_df_pivot.reset_index(inplace=True)

In [20]:
activity_columns = ['avg_strength_12_intercept', 'avg_strength_12_slope', 'avg_strength_12_std_dev', 'avg_strength_3_intercept', 'avg_strength_3_slope', 'avg_strength_3_std_dev', 'avg_strength_6_intercept', 'avg_strength_6_slope', 'avg_strength_6_std_dev', 'avg_strength_9_intercept', 'avg_strength_9_slope', 'avg_strength_9_std_dev', 'betweenness_centrality_12_intercept', 'betweenness_centrality_12_slope', 'betweenness_centrality_12_std_dev', 'betweenness_centrality_3_intercept', 'betweenness_centrality_3_slope', 'betweenness_centrality_3_std_dev', 'betweenness_centrality_6_intercept', 'betweenness_centrality_6_slope', 'betweenness_centrality_6_std_dev', 'betweenness_centrality_9_intercept', 'betweenness_centrality_9_slope', 'betweenness_centrality_9_std_dev', 'closeness_centrality_12_intercept', 'closeness_centrality_12_slope', 'closeness_centrality_12_std_dev', 'closeness_centrality_3_intercept', 'closeness_centrality_3_slope', 'closeness_centrality_3_std_dev', 'closeness_centrality_6_intercept', 'closeness_centrality_6_slope', 'closeness_centrality_6_std_dev', 'closeness_centrality_9_intercept', 'closeness_centrality_9_slope', 'closeness_centrality_9_std_dev', 'degree_centrality_12_intercept', 'degree_centrality_12_slope', 'degree_centrality_12_std_dev', 'degree_centrality_3_intercept', 'degree_centrality_3_slope', 'degree_centrality_3_std_dev', 'degree_centrality_6_intercept', 'degree_centrality_6_slope', 'degree_centrality_6_std_dev', 'degree_centrality_9_intercept', 'degree_centrality_9_slope', 'degree_centrality_9_std_dev', 'num_relationships_12_intercept', 'num_relationships_12_slope', 'num_relationships_12_std_dev', 'num_relationships_3_intercept', 'num_relationships_3_slope', 'num_relationships_3_std_dev', 'num_relationships_6_intercept', 'num_relationships_6_slope', 'num_relationships_6_std_dev', 'num_relationships_9_intercept', 'num_relationships_9_slope', 'num_relationships_9_std_dev', 'number_of_comments_issues_12_intercept', 'number_of_comments_issues_12_slope', 'number_of_comments_issues_12_std_dev', 'number_of_comments_issues_3_intercept', 'number_of_comments_issues_3_slope', 'number_of_comments_issues_3_std_dev', 'number_of_comments_issues_6_intercept', 'number_of_comments_issues_6_slope', 'number_of_comments_issues_6_std_dev', 'number_of_comments_issues_9_intercept', 'number_of_comments_issues_9_slope', 'number_of_comments_issues_9_std_dev', 'number_of_issues_12_intercept', 'number_of_issues_12_slope', 'number_of_issues_12_std_dev', 'number_of_issues_3_intercept', 'number_of_issues_3_slope', 'number_of_issues_3_std_dev', 'number_of_issues_6_intercept', 'number_of_issues_6_slope', 'number_of_issues_6_std_dev', 'number_of_issues_9_intercept', 'number_of_issues_9_slope', 'number_of_issues_9_std_dev', 'number_of_pr_12_intercept', 'number_of_pr_12_slope', 'number_of_pr_12_std_dev', 'number_of_pr_3_intercept', 'number_of_pr_3_slope', 'number_of_pr_3_std_dev', 'number_of_pr_6_intercept', 'number_of_pr_6_slope', 'number_of_pr_6_std_dev', 'number_of_pr_9_intercept', 'number_of_pr_9_slope', 'number_of_pr_9_std_dev', 'number_of_revisions_12_intercept', 'number_of_revisions_12_slope', 'number_of_revisions_12_std_dev', 'number_of_revisions_3_intercept', 'number_of_revisions_3_slope', 'number_of_revisions_3_std_dev', 'number_of_revisions_6_intercept', 'number_of_revisions_6_slope', 'number_of_revisions_6_std_dev', 'number_of_revisions_9_intercept', 'number_of_revisions_9_slope', 'number_of_revisions_9_std_dev']
regression_df_pivot['turnover'] = regression_df_pivot[activity_columns].sum(axis=1) == 0
regression_df_pivot['turnover'] = regression_df_pivot['turnover'].apply(lambda x: 'died' if x else 'active')

In [21]:
df = regression_df_pivot

In [22]:
df[df.user == 'minemR']

,user,project,current_month,avg_strength_12_intercept,avg_strength_12_slope,avg_strength_12_std_dev,avg_strength_3_intercept,avg_strength_3_slope,avg_strength_3_std_dev,avg_strength_6_intercept,...,number_of_revisions_3_intercept,number_of_revisions_3_slope,number_of_revisions_3_std_dev,number_of_revisions_6_intercept,number_of_revisions_6_slope,number_of_revisions_6_std_dev,number_of_revisions_9_intercept,number_of_revisions_9_slope,number_of_revisions_9_std_dev,turnover


In [23]:
df = df.sort_values(by=['user', 'project', 'current_month']).reset_index(drop=True)
cleaned_data = []

for (developer, project), group in df.groupby(['user', 'project']):
    death_months = group[group['turnover'] == 'died']['current_month']
    
    if not death_months.empty:
        for death_month in death_months:
            pre_death_range = pd.date_range(
                end = death_month - pd.DateOffset(months=1),
                periods = 11,
                freq='MS'
            )
            
            last_month = death_month - pd.DateOffset(months=12)

            for _, row in group.iterrows():
                if row['current_month'] in pre_death_range:
                    row['turnover'] = 'pre-death'
                
                elif row['current_month'] == last_month:
                    row['turnover'] = 'last_month'

                cleaned_data.append(row)
    else:
        cleaned_data.extend(group.to_dict('records'))

df_cleaned = pd.DataFrame(cleaned_data)

In [24]:
df_cleaned = df_cleaned.sort_values(by=['user', 'project', 'current_month']).reset_index(drop=True)
df_cleaned = df_cleaned.groupby(['user', 'project']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

/var/folders/c1/3ypglz8x6bx6_n9y6760l6n9h4gtsy/T/ipykernel_40368/967631428.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df_cleaned.groupby(['user', 'project']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)


In [25]:
df_cleaned[df_cleaned.turnover == 'died'].sort_values(by = 'current_month', ascending=False)

,user,project,current_month,avg_strength_12_intercept,avg_strength_12_slope,avg_strength_12_std_dev,avg_strength_3_intercept,avg_strength_3_slope,avg_strength_3_std_dev,avg_strength_6_intercept,...,number_of_revisions_3_intercept,number_of_revisions_3_slope,number_of_revisions_3_std_dev,number_of_revisions_6_intercept,number_of_revisions_6_slope,number_of_revisions_6_std_dev,number_of_revisions_9_intercept,number_of_revisions_9_slope,number_of_revisions_9_std_dev,turnover
21311,VinceBa,freecad_freecad,2024-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
21315,VinceBa,freecad_freecad,2024-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
21320,VinceBa,freecad_freecad,2024-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
21319,VinceBa,freecad_freecad,2024-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
21318,VinceBa,freecad_freecad,2024-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48483,j-wiedemann,freecad_freecad,2015-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
48482,j-wiedemann,freecad_freecad,2015-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
81504,yorikvanhavre,freecad_freecad,2015-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
81503,yorikvanhavre,freecad_freecad,2015-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,died
